## 7.2 整型变量和操作符

### 整数大小

整数可以是不同的大小，并且可以是有符号或无符号的。下表总结了可用的不同整数类型。

 declaration | size, bits | minimum value | maximum value | in stdint.h 
 ------------|------------|------------   | ------------  | ------------
char                               |  8 |-128   | 127   | int8_t
short int (in 16-bit systems: int) | 16 |-32768 | 32767 | int16_t
int (in 16-bit systems: long int)  | 32 | -$2^{31}$ | $2^{31}$-1| int32_t
long long or int64_t<br>MS compiler: `__int64`<br>64-bit Linux: long int | 64 | -$2^{63}$ | $2^{63}$-1 | int64_t
unsigned char                      | 8  | 0     | 255   | uint8_t
unsigned short int<br>in 16-bit systems: unsigned int | 6 | 0 | 65535 | uint16_t
unsigned int<br>in 16-bit systems: unsigned long      | 32 | 0 | $2^{32}$-1 | uint32_t
unsigned long long or uint64_t<br>MS compiler: unsigned `__int64`<br>64-bit Linux: unsigned long int| 64 | 0 | $2^{64}$-1 | uint64_t
**Table 7.1** 不同整数类型的大小

就如上表所看到的，不幸的是，对于不同的平台，声明特定大小的整数的方式是不同的。
**如果标准头文件`stdint.h`或`inttypes.h`存在，那么建议将其使用它们来定义特定大小的整数类型，以达到可移植的目的。**

无论整数的大小如何，整数运算在大多数情况下都很快。
**但是，如果超过平台提供的最大寄存器大小的整数， 效率不高。**
换句话说，在16位系统中使用32位整数，或在32位系统中使用64位整数是低效的，尤其是如果代码涉及乘法或除法。

如果您声明了一个int，编译器将始终选择最有效的整数大小，而不指定大小。
较小尺寸的整数（char，short int）效率仅仅稍低一些。
在许多情况下，编译器会在计算时将这些类型转换为默认大小的整数，然后仅使用结果的低8位或16位。
您可以假设该类型转换需要零个或一个时钟周期。
在64位系统中，只要不做除法，32位整数和64位整数的效率之间的差异只有很小的差别。

建议在下面情况下使用缺省大小的整数：
- 大小无关紧要
- 并且不存在溢出风险

例如，简单变量，循环计数器等。

在大型数组中，推荐首选使用够用的最小类型整数，以便更好地使用数据缓存。
对于位字段，除8,16,32和64位以外的其他的位，效率较低。
在64位系统中，如果应用程序可以使用额外的位，则可以使用64位整数。

无符号整数类型`size_t`在32位系统中为32位，在64位系统中为64位。
此类型一般用于表示数组大小和数组索引，可以确保永远不会发生溢出时，即使对于大于2GB的数组。

在考虑某个整数大小是否足够满足特定目的时，你**必须考虑中间计算是否会导致溢出。**
例如，在表达式`a=(b*c)/d`中，即使a，b，c和d都会低于最大值，也可能发生`(b*c)`溢出。
编译器没有自动检查整数溢出。


### 有符号及无符号整数

在大多数情况下，使用带符号和无符号整数的速度没有什么区别。但是有一些值得注意的情况：

- 除以常量：**整数被常量除的时候，无符号整数比有符号要快**（请参见第141页）。这也适用于取模运算符`%`。


- 对于大多数指令集，**带符号的转换为浮点的速度比无符号整数的速度快**（请参见第145页）。


- **溢出行为在有符号和无符号变量上表现不同**。
  - 无符号变量的溢出产生的小的正整数结果。
  - 有符号变量的溢出产生的没有被正式定义过。一般行为是将正溢出环绕转换为负值，但编译器可能会根据溢出不发生的假设进行优化，把溢出的分支处理给去除。


有符号和无符号整数之间的转换是无成本的。 这只不过是同一个位数据进行不同解释的问题。
负整数转换为无符号整数时，将被当作为非常大的正数。


```cpp
// Example 7.4. Signed and unsigned integers
int a, b;
double c;
b = (unsigned int)a / 10; // Convert to unsigned for fast division
c = a * 2.5; // Use signed when converting to double
```

**
在例7.4中，我们将`a`转换为无符号，以使分区更快。
当然，这只有在确定`a`永远不会为负的情况下才有效。
最后一行隐含地将`a`转换为`double`，然后与`double`常数`2.5`相乘。在这里当要转换成double时，我们更倾向于使用有符号数。
**

请务必**不要在比较中混合使用有符号和无符号整数**，例如`<`。 比较有符号与无符号整数的结果不明确，可能会产生不希望的结果

### 整数运算

整数操作通常非常快。
简单的整数运算，例如加法，减法，比较，位操作和移位操作，在大多数微处理器上只需要一个时钟周期。

乘法和除法需要更长的时间。
整数乘法在Pentium 4处理器上需要11个时钟周期，在大多数其他微处理器上需要3-4个时钟周期。
**整数除法需要40-80个时钟周期，具体取决于微处理器。**
整数除法在AMD处理器上，整数宽度越小，速度越快。但在英特尔处理器上不是这样。
有关指令等待时间的详细信息在手册4：“指令表”中列出。
有关如何加速乘法和除法的提示分别在第140页和第141页给出。

### 自增和自减运算

前加运算符 `++i` 和后加运算符 `i++` 的速度与加法速度一样快。
当用于简单地自增一个整数变量时，使用前加还是后加没有区别。效果完全相同。

例如：对于`(i = 0; i < n; i++)` 与 `(i = 0; i < n; ++i)`相同。
**但是，当在表达式中使用时，效率可能会有所不同。**
**例如，`x = array[i++]`比`x = array[++i]`更高效**，因为在后一种情况下，数组元素地址的计算必须等待`i`的新值，这会延迟大约两个时钟周期后，`x`才可用。
显然，如果将前增量更改为后增量，则必须调整`i`的初始值。

**还有一种情况，前加比后加更有效。**
例如，在`a = ++b`的情况下，编译器会认识到，在这个语句之后，a和b的值是相同的，这样它就可以为这两者使用相同的寄存器，而表达式`a = b++;` 将使a和b的值不同，它们不能使用相同的寄存器。

这里所说的有关自增运算符的所有内容，也适用于整数变量的自减运算符。